# Breakout
En este proyecto recrearemos el juego clasico de breakout con algunas modificaciones para que sea mas divertido y a su vez siga patrones de diseño.



### Singleton
Utilizamos el patron singleton para configurar el tamaño de la pantalla y y el tamaño de los objetos del juego. Prevenimos que exista otra estancia de esta misma configuración

In [17]:
class GameConfig:
    # Declaramos una variable estatica privada que no puede ser modificada.
    __instance = None

    # Creamos un metodo estatico que nos permita acceder a esta instancia.
    @staticmethod
    def get_instance():
        if GameConfig.__instance is None:
            GameConfig()
        return GameConfig.__instance
    def __init__(self):
        # Declaramos las variables de configuracion del juego y aseguramos que no existan otras instancias de la clase.
        if GameConfig.__instance is not None:
            raise Exception("Esta clase es un singleton, no se puede crear otra instancia.")
        else:
            GameConfig.__instance = self
        
        # Configuracion del juego por defecto
        self.width = 1920
        self.height = 1080
        self.fps = 60
        self.ball_size = 10

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def set_fps(self, fps):
        self.fps = fps

    def set_ball_size(self, ball_size):
        self.ball_size = ball_size

### Flyweight y Factory
Utilizamos el patron flyweight para reducir el uso de memoria con los sprites utilizados en el proyecto

In [18]:
import sys
import pygame

from typing import Tuple, Dict
from dataclasses import dataclass

# Definimos las para la textura del juego ya que es constante y no cambia
@dataclass
class BallType:
    texture: pygame.Surface

# Definimos la clase Ball que representa una bola en el juego, con su textura y posición
class Ball:
    def __init__(self, ball_type: BallType, x: int, y: int, velocity: Tuple[int, int]):
        self.ball_type = ball_type
        self.x = x
        self.y = y
        self.velocity = velocity
        self.is_active = True

    # Actualizamos la posición de la bola en el juego
    def move(self):
        dx, dy = self.velocity
        self.x += dx
        self.y += dy

    def get_position(self) -> Tuple[int, int]:
        return self.x, self.y
    
    
# Definimos la clase BallManager que maneja las bolas en el juego
class BallManager:
    def __init__(self):
        self._balls: list[Ball] = []

    def add_ball(self, ball: "Ball") -> None:
        self._balls.append(ball)          

    def remove_ball(self, ball: "Ball") -> None:
        if ball in self._balls:
            self._balls.remove(ball)

    def get_balls(self) -> list["Ball"]:
        return self._balls      

# Definimos la clase BallTypeFactory que maneja la creación de los tipos de bolas           
class BallTypeFactory:
    _ball_types: Dict[str, BallType] = {}  # Cache para los tipos de bolas

    @classmethod
    def create_ball_type(cls, texture_path: str) -> BallType:
        # Crea un nuevo tipo de bola si no existe en el cache
        # Si existe, lo devuelve directamente
        try:
            if texture_path not in cls._ball_types:
                texture = pygame.image.load(texture_path)
                cls._ball_types[texture_path] = BallType(texture)
            return cls._ball_types[texture_path]
        except pygame.error as e:
            raise ValueError(f"Failed to create BallType: {e}")




### Command
Utilizamos el patron command para simplificar y separar el control del usuario 

In [19]:
from abc import ABC, abstractmethod

# Clase commnad abstracta que define la interfaz para los comandos
class Command(ABC):
    @abstractmethod
    def execute(self) -> None:
        pass

# Clase Player que representa al jugador recibir comandos de movimiento
class Player:
    def __init__(self, x: int, y: int, width: int):
        self.x = x
        self.y = y
        self.width = width

    def move_left(self, distance: int) -> None:
        new_x = self.x - distance
        if new_x > 0:
            self.x = new_x

    def move_right(self, distance: int) -> None:
        new_x = self.x + distance
        if new_x < GameConfig.get_instance().width:
            self.x = new_x

    def get_position(self) -> Tuple[int, int]:
        return self.x, self.y


# Comandos concretos que implementan la interfaz Command
class MoveLeftCommand(Command):
    def __init__(self, player: Player, distance: int):
        self.player = player
        self.distance = distance

    def execute(self) -> None:
        self.player.move_left(self.distance)

class MoveRightCommand(Command):
    def __init__(self, player: Player, distance: int):
        self.player = player
        self.distance = distance

    def execute(self) -> None:
        self.player.move_right(self.distance)

# Clase InputHandler que maneja la entrada del usuario
class InputHandler:
    def __init__(self):
        self.commands: Dict[int, Command] = {}

    def bind_input(self, key: int, command: Command) -> None:
        self.commands[key] = command

    def handle_input(self, event) -> None:
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.KEYDOWN:
            if event.key in self.commands:
                self.commands[event.key].execute()         


### Builder

Para crear ladrillos utilizamos el patrón builder que nos permite construir ladrillos practicamente identicos.

In [20]:
# Definimos la clase Brick que representa un ladrillo en el juego
class Brick:
    def __init__(self, x=None, y=None, width=None, height=None):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.is_active = True

    def get_position(self) -> Tuple[int, int]:
        return self.x, self.y

    def get_size(self) -> Tuple[int, int]:
        return self.width, self.height
    
    # Desactiva el ladrillo cuando es golpeado
    def hit(self) -> None:
        self.is_active = False

# Definimos la clase BrickBuilder que implementa el patrón Builder para crear ladrillos
class BrickBuilder:
    def __init__(self):
        self.brick = Brick()
    
    def set_position(self, x: int, y: int) -> 'BrickBuilder':
        self.brick.x = x
        self.brick.y = y
        return self
    
    def set_size(self, width: int, height: int) -> 'BrickBuilder':
        self.brick.width = width
        self.brick.height = height
        return self
    
    def build(self) -> Brick:
        built_brick = self.brick
        self.brick = Brick() # Reinicia el constructor para crear un nuevo ladrillo
        return built_brick

# Definimos la clase BrickWall que representa una pared de ladrillos en el juego
class BrickWall:
    def __init__(self, width: int, height: int, brick_width: int, brick_height: int, spacing: int = 5):
        self.bricks = []
        self.width = width
        self.height = height
        self.brick_width = brick_width
        self.brick_height = brick_height
        self.spacing = spacing
        self.builder = BrickBuilder()
        self.create_wall()

    # Crea una pared de ladrillos en el juego
    # Separa los ladrillos por el espacio definido
    def create_wall(self) -> None:
        for y in range(0, self.height, self.brick_height + self.spacing):
            for x in range(0, self.width, self.brick_width + self.spacing):
                brick = (
                    self.builder
                    .set_position(x, y)
                    .set_size(self.brick_width, self.brick_height)
                    .build()
                )
                self.bricks.append(brick)

                
    def get_bricks(self) -> list:
        return self.bricks
    
    def get_active_bricks(self) -> list:
        return [brick for brick in self.bricks if brick.is_active]


### Inicializacion del juego/cliente con la libreria pygame 

In [21]:
from pygame.locals import QUIT, KEYDOWN, K_ESCAPE, K_LEFT, K_RIGHT
import random

# Inicializamos la configuración del juego
config = GameConfig.get_instance()
config.set_width(800)
config.set_height(600)
config.set_fps(30)
config.set_ball_size(5)

pygame.init()
screen = pygame.display.set_mode((config.width, config.height))
pygame.display.set_caption("BREAKOUT BY TUTIS")

# Definimos el jugador y su posición inicial
player = Player(config.width // 2, config.height - config.height//10, config.width // 8)

# Cargamos la textura de la bola
ball_type = BallTypeFactory.create_ball_type("../textures/ball_texture.png")

# Creamos la bola inicial y la añadimos al gestor de bolas
ball_manager = BallManager()
initial_ball = Ball(ball_type, player.x + player.width // 2, player.y - config.ball_size, (5, -5))
ball_manager.add_ball(initial_ball)

# Creamos la pared de ladrillos
brick_wall = BrickWall(
    width=config.width,
    height=config.height // 3,
    brick_width=config.width // 6,
    brick_height=config.height // 16,
    spacing=5
)

brick_builder = BrickBuilder()

# Asignamos los inputs del jugador
input_handler = InputHandler()
input_handler.bind_input(K_LEFT, MoveLeftCommand(player, player.width//2))
input_handler.bind_input(K_RIGHT, MoveRightCommand(player, player.width//2))

# Creamos la ventana del juego y el bucle principal
running = True
while running:
    # Limitamos la velocidad de fotogramas
    pygame.time.Clock().tick(config.fps)

    for event in pygame.event.get():
        # Manejamos los eventos de entrada
        input_handler.handle_input(event)
        if event.type == QUIT or (event.type == KEYDOWN and event.key == K_ESCAPE):
            running = False

    # Actualizamos la pantalla
    screen.fill((0, 0, 0))

    # Dibujamos el jugador
    pygame.draw.rect(screen, (255, 255, 255), (player.x, player.y, player.width, player.width//4))

    balls_to_remove = [] # Lista para almacenar bolas que deben ser eliminadas

    # Actualizamos la posición de las bolas y verificamos colisiones
    for ball in ball_manager.get_balls():
        ball_rect = pygame.Rect(ball.x - config.ball_size, ball.y - config.ball_size, config.ball_size*2, config.ball_size*2)
        screen.blit(ball.ball_type.texture, ball_rect)

        # Verificamos colisiones con los ladrillos
        for brick in brick_wall.get_bricks():
            if brick.is_active:
                brick_rect = pygame.Rect(brick.x, brick.y, brick.width, brick.height)
                if brick_rect.colliderect(ball_rect):
                    brick.hit()
                    ball.velocity = (ball.velocity[0], -ball.velocity[1])
                    new_ball = Ball(
                        ball_type,
                        ball.x + random.randint(-config.ball_size, config.ball_size) * 2,
                        ball.y + random.randint(-config.ball_size, config.ball_size) * 2,
                        (-ball.velocity[0], ball.velocity[1])
                    )
                    ball_manager.add_ball(new_ball)
                    break

        # Verificamos colisiones con el jugador
        player_rect = pygame.Rect(player.x, player.y, player.width, player.width//4)
        if player_rect.collidepoint(ball.x, ball.y):
            ball.velocity = (ball.velocity[0], -abs(ball.velocity[1]))

        # Verificamos colisiones con los bordes de la pantalla
        if ball.x - config.ball_size <= 0 or ball.x + config.ball_size >= config.width:
            ball.velocity = (-ball.velocity[0], ball.velocity[1])
        if ball.y - config.ball_size <= 0:
            ball.velocity = (ball.velocity[0], -ball.velocity[1])
        if ball.y + config.ball_size >= config.height:
            balls_to_remove.append(ball)

        # Actualizamos la posición de la bola
        ball.move()

    # Eliminamos las bolas que han salido de la pantalla
    for ball in balls_to_remove:
        ball_manager.remove_ball(ball)

    # Verificamos si el juego ha terminado
    if not ball_manager.get_balls():
        running = False
    if not brick_wall.get_active_bricks():
        running = False

    # Dibujamos ladrillos
    for brick in brick_wall.get_bricks():
        if brick.is_active:
            brick_rect = pygame.Rect(brick.x, brick.y, brick.width, brick.height)
            pygame.draw.rect(screen, (255, 0, 0), brick_rect)
            pygame.draw.rect(screen, (0, 0, 0), brick_rect, 1)

    pygame.display.flip()

pygame.quit()
